In [ ]:
# task 3 - IMAGE CAPTIONING

import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout, Add
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt

base_model = InceptionV3(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

def extract_features(image_path):
    img = Image.open(image_path).resize((299, 299))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = model.predict(img)
    return features

with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

max_length = 34  

def generate_caption(model, image_path):
    features = extract_features(image_path)
    caption = "startseq"
    
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([caption])[0]
        sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=max_length)
        y_pred = model.predict([features, sequence], verbose=0)
        word_index = np.argmax(y_pred)
        word = tokenizer.index_word.get(word_index, '')
        if word == 'endseq':
            break
        caption += ' ' + word
    
    return caption.replace("startseq", "").replace("endseq", "").strip()

caption_model = tf.keras.models.load_model("caption_model.h5")

image_path = "example.jpg"
predicted_caption = generate_caption(caption_model, image_path)

plt.imshow(Image.open(image_path))
plt.axis("off")
plt.title(predicted_caption)
plt.show()